## Ce notebook sert à tout simplement à ajouter au tableau des articles du BSPE une colonne qui distingue la catégorie de l'article 

* Communication
* Mémoires
* Discussion
* Administration

In [2]:
import requests
import csv
import pandas as pd

In [10]:
with open ("Contributions_TAB_BSPE.tsv") as BSPE:
    df = pd.read_csv(BSPE,sep='\t')

In [11]:
df

,Unnamed: 0,article_id,article_auteur,article_titre,article_page,revue_annee,revue_numero
0,0,BSPE-1-1,LAVERAN (Alphonse),Discours d’ouverture.,1,1908,1
1,1,BSPE-1-15,"MARTIN (Louis), DARRÉ",Sur les symptômes nerveux du début de la malad...,15,1908,1
2,2,BSPE-1-18,SERGENT (ED.),La fièvre méditerranéenne en Algérie. Note pré...,18,1908,1
3,3,BSPE-1-22,REMY (F. J.),Le Debab dans la région de Barika.,22,1908,1
4,4,BSPE-1-25,"SERGENT(ED.),SERGENT(ÉT.), LEDOUX (ED.)",Le Debab dans la région de la Zousfana (SudOra...,25,1908,1
...,...,...,...,...,...,...,...
2438,2438,BSPE-14-662,ROUBAUD (E.),Le bien-être dans le Paludisme et les maladies...,662,1921,14
2439,2439,BSPE-14-666,JOUVEAU-DUBREUIL (H.),Le Béribéri au Setchouan,666,1921,14
2440,2440,BSPE-14-685,"DONATIEN (A.), SERGENT (ED., ET.), PARROT (L.)",Le Paludisme en Corse. Etude épidémiologique.,685,1921,14
2441,2441,BSPE-14-710,LAVIER (G.),Flagellés parasites intestinaux du campagnol i...,710,1921,14


In [19]:
import re
for entree in df["article_titre"]: 
    if re.match("Discussion", entree):
            df["article_section"] = "Discussion"###Ne fonctionnepas _ 